---------------------------------------

Loading data (Uniprot / Mgnify / NCBI) from a IDs input
---------------------------------------

In [ ]:
import requests
from Bio import Entrez

Entrez.email = "justin.neo2@gmail.com"   # required by NCBI
OUTPUT_FILE = "erickson_petasevariant.fasta"
#Some entries like R4YKL9_OLEAN, E9LVI0_THEFU, Q47RJ6_THEFY are UniProt accession + organism suffix.
#The REST API expects only the accession (R4YKL9, E9LVI0, Q47RJ6) — not the _XXXX part.
# read IDs from file, pre-clean the     IDs to remove any .1 .2 etc. 
#some IDs are in a different format than the one REST needs, on uniprot use the entry name strictly  
#
with open("petase_db/erickson_petasevariant_id.txt") as f:
    ids = [line.strip() for line in f if line.strip()]

#UNIPROT 
def fetch_uniprot(uid):
    url = f"https://rest.uniprot.org/uniprotkb/{uid}.fasta"
    r = requests.get(url)
    return r.text if r.status_code == 200 else None

#MGNIFY
def fetch_mgnify(mid):
    url = f"https://www.ebi.ac.uk/metagenomics/api/latest/sequence/{mid}.fasta"
    r = requests.get(url)
    return r.text if r.status_code == 200 else None

#ENTREZ NCBI 
def fetch_ncbi(nid):
    for db in ["protein", "nuccore"]:
        try:
            handle = Entrez.efetch(db=db, id=nid, rettype="fasta", retmode="text")
            fasta = handle.read()
            if fasta.strip():
                return fasta
        except Exception:
            continue
    return None

with open(OUTPUT_FILE, "w") as out:
    for seq_id in ids:
        print(f"Fetching {seq_id} ...")
        fasta = None
        #if seq_id.startswith(("A0", "Q", "P", "E")) or seq_id == "MPZ00478" or seq_id == "MBO9532528" or seq_id == "MBX2857432" or seq_id == "MBX3625601":
        #    fasta = fetch_uniprot(seq_id)
        #elif seq_id.startswith(("MGYP", "MBX", "MBO", "MPZ")):
        #    fasta = fetch_mgnify(seq_id)
        #else:
        #    fasta = fetch_ncbi(seq_id)
        fasta = fetch_ncbi(seq_id)
        if fasta is None:
            print(f"⚠️ Could not fetch {seq_id}")
        else:
            out.write(fasta)

print(f"✅ Done! Sequences saved in {OUTPUT_FILE}")

Unified PETase seq from Uniprotec + Pazy

100: 464

cd-hit 0.99: 489 ->  457

0.98: 446

0.97: 434

0.96: 422

95: 413 


CaPETase mutation adding

In [17]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

def apply_mutations_to_fasta(input_fasta, output_fasta, mutations, offset=27):
    """
    Apply a list of mutations (e.g. ["S121E", "D186H"]) to a FASTA sequence.

    Parameters:
        input_fasta (str): path to input fasta (with WT sequence)
        output_fasta (str): path to save mutated fasta
        mutations (list): list of mutation strings (e.g. ["S121E", "D186H"])
        offset (int): numbering offset (e.g. 27 if sequence is signal peptide–trimmed)
    """

    # Read first sequence from fasta
    record = next(SeqIO.parse(input_fasta, "fasta"))
    seq_list = list(str(record.seq))
    original_seq = str(record.seq)

    applied = []
    for mut in mutations:
        wt_res = mut[0]            # first letter
        new_res = mut[-1]          # last letter
        pos = int(mut[1:-1])       # number in between
        idx = pos - offset - 1     # adjust and convert to 0-based

        if idx < 0 or idx >= len(seq_list):
            applied.append((mut, "⚠️ out of range"))
            continue

        if seq_list[idx] != wt_res:
            applied.append((mut, f"⚠️ mismatch: found {seq_list[idx]} at pos {pos}, expected {wt_res}"))
        else:
            seq_list[idx] = new_res
            applied.append((mut, "ok"))

    # Make new record
    mutated_seq = "".join(seq_list)
    new_id = record.id + "_" + "_".join(mutations)
    new_record = SeqRecord(Seq(mutated_seq), id=new_id, description="Mutated variant")

    # Write FASTA
    SeqIO.write(new_record, output_fasta, "fasta")

    return applied, mutated_seq

# Example usage:
mutations = ["T250N"]
applied, mutated_seq = apply_mutations_to_fasta(
    "petase_db/wtcapetase.fasta",
    "CaPETase_T250N.fasta",
    mutations,
    offset=0
)

print("Applied mutations:")
for m in applied:
    print(m)

BhrPETase LCC Son Mutations Adding

In [23]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd

#mutations
mutations_bhr = [
    "W104L","W104S","W104C","W104H","W104D","W104R","W104G",
    "H164L","H164S","H164E","H164Q","H164F",
    "M166L","M166S","M166D","M166F",
    "W190L","W190M","W190S","W190H","W190D",
    "H191L","H191M","H191S","H191D","H191Y",
    "F243I","F243S","F243T","F243D","F243N","F243G",
    "H218S","H218S/F222I"
]
mutations_son = [
    "S121D",
    "S121E",
    "D186H",
    "D186F",
    "D186I",
    "D186L",
    "D186V",
    "P181A",
    "P181G",
    "P181S"
]

mutations_lcc = [
    "T96M",
    "Y127G",
    "F243I",
    "F243W",
    "N246D",
    "N246M",
    "D238C/S283C",
    "F243I/D238C/S283C",
    "F243W/D238C/S283C",
    "F243I/D238C/S283C/T96M",
    "F243I/D238C/S283C/Y127G",
    "F243I/D238C/S283C/N246D",
    "F243I/D238C/S283C/N246M",
    "F243W/D238C/S283C/T96M",
    "F243W/D238C/S283C/Y127G",
    "F243W/D238C/S283C/N246D",
    "F243W/D238C/S283C/N246M"
]
# ---- Input ----
wt_fasta = "petase_db/sequences/wtbhrpetase.fasta"   # WT FASTA
output_fasta = "bhrtest.fasta"

# ---- Load WT sequence ----
wt_record = SeqIO.read(wt_fasta, "fasta")
wt_seq = list(str(wt_record.seq))

OFFSET = 0  # negative offset

def apply_mutations(wt_seq, mutations_str):
    seq = wt_seq.copy()
    for mut in mutations_str.replace("+", "/").split("/"):
        if not mut:
            continue
        wt_aa = mut[0]
        pos = int(mut[1:-1])
        pos += OFFSET   # << now applies negative offset if set
        new_aa = mut[-1]
        if pos-1 >= len(seq) or pos-1 < 0:
            raise ValueError(f"Mutation {mut} out of range for sequence length {len(seq)} (index {pos-1})")
        if seq[pos-1] != wt_aa:
            print(f"⚠️ Warning: expected {wt_aa} at {pos}, found {seq[pos-1]}")
        seq[pos-1] = new_aa
    return "".join(seq)

# ---- Generate all variants ----
records = []
for mut in mutations_bhr:
    new_seq = apply_mutations(wt_seq, mut)
    record = SeqRecord(Seq(new_seq), id=f"bhrtest_{mut}", description=mut)
    records.append(record)

# ---- Write output FASTA ----
SeqIO.write(records, output_fasta, "fasta")
print(f"✅ Wrote {len(records)} variants to {output_fasta}")

✅ Wrote 34 variants to bhrtest.fasta


PETase mutation scoring for IsPETase

In [ ]:
from Bio import SeqIO
import re
import pandas as pd

def sequences_position_table(input_fasta, mutations, offset=27):
    """
    Build a table with columns as mutation positions and rows as sequences,
    showing the amino acid present at each position.

    Parameters:
        input_fasta (str): path to FASTA containing one or more sequences
        mutations (list): list of mutation strings (e.g., ["S160", "D206A", "W159[H/A]"])
        offset (int): numbering offset (subtract this many from the positions)

    Returns:
        pd.DataFrame: table with mutations as columns and sequences as rows
    """
    # Regex to parse mutation
    pattern = re.compile(r"([A-Z])?(\d+)([A-Z]|\[[A-Z/]+\])?")

    # Parse mutation positions once
    mut_info = []
    for mut in mutations:
        m = pattern.match(mut)
        if not m:
            continue
        _, pos, _ = m.groups()
        pos = int(pos) - offset
        idx = pos - 1
        mut_info.append((mut, idx))

    # Collect rows for each sequence
    rows = []
    row_ids = []

    for record in SeqIO.parse(input_fasta, "fasta"):
        seq = str(record.seq)
        row_ids.append(record.id)

        row_vals = []
        for mut, idx in mut_info:
            if 0 <= idx < len(seq):
                row_vals.append(seq[idx])
            else:
                row_vals.append("NA")  # if index out of range
        rows.append(row_vals)

    # Build DataFrame
    col_labels = [mut for mut, _ in mut_info]
    df = pd.DataFrame(rows, columns=col_labels, index=row_ids)
    return df


# Example usage:
mutations = [
    "S160", "D206", "H237", "T88", "W159", "W185", "I208", "S214", "Y87", "M161",
    "C203", "C239", "C273", "C289", "S160A", "Y87A", "M161A", "T88A", "W159[H/A]",
    "W185A", "I208A", "S214H", "C203S", "C239S", "S160", "D206", "H237", "Y87",
    "M161", "W185", "I208", "T88", "A89", "W159", "I232", "N233", "S236", "S238F",
    "N241", "N244", "S245", "N246", "R280", "S160A", "D206A", "H237A", "Y87A",
    "M161A", "W185A", "I208A", "W159[H/A]", "S238F", "N241A", "R280A", "C203A",
    "C239A", "R90A", "L117F", "I208F", "R280A", "W159H", "S238F", "L117F", "R280A",
    "T88M", "Q119G", "N233C", "S238I", "N241D", "S282C"
]

df = sequences_position_table("petase_db/sequences/all_petase_variants.fasta", mutations, offset=27)

print(df)
df.to_csv("petase_mutation_table_allseqs.csv")

Mutation scoring for CaPETase variants

In [ ]:
from Bio import SeqIO
import pandas as pd

def sequences_position_table(input_fasta, positions):
    """
    Build a DataFrame where columns are positions and rows are sequences,
    showing the amino acid present at each position.

    Parameters:
        input_fasta (str): path to FASTA with one or more sequences
        positions (list[int]): list of 1-based positions to check

    Returns:
        pd.DataFrame
    """
    rows = []
    row_ids = []

    for record in SeqIO.parse(input_fasta, "fasta"):
        seq = str(record.seq)
        row_ids.append(record.id)

        row_vals = []
        for pos in positions:
            if 1 <= pos <= len(seq):
                row_vals.append(seq[pos-1])  # convert to 0-based index
            else:
                row_vals.append("NA")  # if position is out of range
        rows.append(row_vals)

    df = pd.DataFrame(rows, columns=[str(p) for p in positions], index=row_ids)
    return df


# Example usage:
positions = [169, 246, 215, 194, 223, 227, 101, 102, 103,
             107, 133, 168, 170, 195, 196, 217, 247, 250]

df = sequences_position_table("petase_db/sequences/capetasevariants.fasta", positions)

print(df)
df.to_csv("catase_positions_table.csv")

In [ ]:
from Bio import SeqIO
import pandas as pd
import re

def sequences_mutation_table(input_fasta, mutations, offset=0):
    """
    Build a DataFrame where columns are mutation strings and rows are sequences,
    showing the amino acid present at each (offset-adjusted) position.

    Parameters:
        input_fasta (str): path to FASTA with one or more sequences
        mutations (list[str]): mutation strings (e.g., "S160", "D206A", "W159[H/A]")
        offset (int): numeric offset to apply to positions (default=0)

    Returns:
        pd.DataFrame
    """
    # Regex to parse strings like S160, D206A, W159[H/A]
    pattern = re.compile(r"([A-Z])?(\d+)([A-Z]|\[[A-Z/]+\])?")

    # Parse mutation strings into (label, adjusted pos)
    mut_info = []
    for mut in mutations:
        m = pattern.match(mut)
        if not m:
            continue
        _, pos, _ = m.groups()
        pos = int(pos) + offset  # apply offset
        mut_info.append((mut, pos))

    rows = []
    row_ids = []

    for record in SeqIO.parse(input_fasta, "fasta"):
        seq = str(record.seq)
        row_ids.append(record.id)

        row_vals = []
        for label, pos in mut_info:
            if 1 <= pos <= len(seq):
                row_vals.append(seq[pos-1])  # 1-based to 0-based index
            else:
                row_vals.append("NA")
        rows.append(row_vals)

    df = pd.DataFrame(rows, columns=[label for label, _ in mut_info], index=row_ids)
    return df


# Example usage
mutations = [
    "S160", "D206", "H237", "T88", "W159", "W185", "I208", "S214", "Y87", "M161",
    "C203", "C239", "C273", "C289", "S160A", "Y87A", "M161A", "T88A", "W159[H/A]",
    "W185A", "I208A", "S214H", "C203S", "C239S", "S160", "D206", "H237", "Y87",
    "M161", "W185", "I208", "T88", "A89", "W159", "I232", "N233", "S236", "S238F",
    "N241", "N244", "S245", "N246", "R280", "S160A", "D206A", "H237A", "Y87A",
    "M161A", "W185A", "I208A", "W159[H/A]", "S238F", "N241A", "R280A", "C203A",
    "C239A", "R90A", "L117F", "I208F", "R280A", "W159H", "S238F", "L117F", "R280A",
    "T88M", "Q119G", "N233C", "S238I", "N241D", "S282C"
]

df = sequences_mutation_table("petase_db/sequences/CaPETasevariants.fasta", mutations, offset=9)

print(df)
df.to_csv("capetase_mutation_check_offset9.csv")

fix mutation list agreement across the pdb-fasta for foldx

python fix_mutations_for_pdb.py wtIsPETase_6ilw_Repair.pdb individual_list.txt fixed_list.txt

---------------------
Encoding sequences and structures of variants with esm3
---------------------------------------

In [ ]:
#Load model
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
import torch
import esm
from esm.sdk import client
login(token="removed")

device = (
    "mps" if torch.backends.mps.is_available()
    else "cpu"
)
#Forge API
model=client("esm3-large-2024-08",token="removed")
#test with esmc-6b now instead of 600m to see diff in dimenson of embeddings 

Sequence emebdding esm3

In [ ]:
#SETUP
import torch
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd 
from esm.sdk import client
import os
import pickle
from concurrent.futures import ThreadPoolExecutor
from typing import Sequence
from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
    ProteinType,
)

model = client(
    model="esmc-6b-2024-12", url="https://forge.evolutionaryscale.ai", token="removed"
)

#if esm-6b, change to ith_hidden_layer=55
#reset to: LogitsConfig(return_embeddings=True, return_hidden_states=True)
EMBEDDING_CONFIG = LogitsConfig(
    sequence=True, ith_hidden_layer=20
    
)


def embed_sequence(model: ESM3InferenceClient, sequence: str) -> LogitsOutput:
    protein = ESMProtein(sequence=sequence)
    protein_tensor = model.encode(protein)
    output = model.logits(protein_tensor, EMBEDDING_CONFIG)
    return output


def batch_embed(
    model: ESM3InferenceClient, inputs: Sequence[ProteinType]
) -> Sequence[LogitsOutput]:
    """Forge supports auto-batching. So batch_embed() is as simple as running a collection
    of embed calls in parallel using asyncio.
    """
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(embed_sequence, model, protein) for protein in inputs
        ]
        results = []
        for future in futures:
            try:
                results.append(future.result())
            except Exception as e:
                results.append(ESMProteinError(500, str(e)))
    return results


def read_fasta_fast(fasta_file):
    records = []
    with open(fasta_file) as in_handle:
        for title, seq in SimpleFastaParser(in_handle):
            gene_id = title.split()[0]
            records.append((gene_id, seq, len(seq)))
    return pd.DataFrame(records, columns=["Gene", "Sequence", "Length"])

ESM-3 (large) ~ 98 billion

ESM-3 medium ~ 7 billion

ESM-3 small (open) / “open” ~ 1.4 billion

ESM C (6B variant) ~ 6 billion

ESM C (600M variant) ~ 600 million

ESM C (300M variant) ~ 300 million

---------------------------------------

[TO DO]
[OCT25] Run esm3 seq/struct embedding locally to avoid credits limit (on benchmark DB) using ESM3-open (1.4b parameters)
---------------------------------------

In [ ]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
import torch
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd 
from esm.sdk import client
import os
import pickle
from concurrent.futures import ThreadPoolExecutor
from typing import Sequence
from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
    ProteinType,
)

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to("cpu") # or "cpu"
EMBEDDING_CONFIG = LogitsConfig(return_embeddings=True, return_hidden_states=True)

def embed_sequence(model: ESM3InferenceClient, sequence: str) -> LogitsOutput:
    protein = ESMProtein(sequence=sequence)
    protein_tensor = model.encode(protein)
    output = model.logits(protein_tensor, EMBEDDING_CONFIG)
    return output


def read_fasta_fast(fasta_file):
    records = []
    with open(fasta_file) as in_handle:
        for title, seq in SimpleFastaParser(in_handle):
            gene_id = title.split()[0]
            records.append((gene_id, seq, len(seq)))
    return pd.DataFrame(records, columns=["Gene", "Sequence", "Length"])

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()


input = read_fasta_fast("petase_db/sequences/esm3_seq/wtIsPETase_signaltrim.fasta")
seq = input["Sequence"].iloc[0]
protein = ESMProtein(sequence=seq)
protein_tensor = model.encode(protein)

# Try simplest logits config
config = LogitsConfig(sequence=True)

output = model.logits(protein_tensor, config)
print(output)



---------------------------------------

Run seq embedding on one fasta file

In [ ]:
name = "wtIsPETase_signaltrim"
# === Save outputs to pickle ===
with open(f"petase_db/sequences/esm3_seq/seq_embeddings/{name}_outputs.pkl", "wb") as f:
    pickle.dump(output, f)

with open(f"petase_db/sequences/esm3_seq/seq_embeddings/{name}_meanembeddings.pkl", "wb") as f:
    pickle.dump(all_mean_embeddings, f)

In [ ]:
from esm.sdk.api import LogitsConfig
input = read_fasta_fast("petase_db/sequences/esm3_seq/wtIsPETase_signaltrim.fasta")
seq = input["Sequence"].iloc[0]
protein = ESMProtein(sequence=seq)
protein_tensor = model.encode(protein)

# Try simplest logits config
config = LogitsConfig(sequence=True)

output = model.logits(protein_tensor, config)
print(output)

In [ ]:
input = read_fasta_fast("petase_db/sequences/esm3_seq/wtIsPETase_signaltrim.fasta")
seq = input["Sequence"].iloc[0]
protein = ESMProtein(sequence=seq)
protein_tensor = model.encode(protein)
output = model.logits(protein_tensor, EMBEDDING_CONFIG)


note that all_mean_embeddings is the mean collapsing all the layers

Batch input fasta files to embed

In [ ]:
#for the sequences
for file in os.listdir("petase_db/sequences/esm3_seq"):
    if file.endswith(".fasta"):
        fasta_path = os.path.join("petase_db/sequences/esm3_seq", file)

        # load sequences
        df = read_fasta_fast(fasta_path)
        seq = df["Sequence"].to_list()

        # run embeddings
        output = batch_embed(model, seq)

        # summarize mean embeddings
        all_mean_embeddings = [
            torch.mean(o.hidden_states, dim=0).squeeze().cpu()
            for o in output
        ]

        name = os.path.splitext(file)[0]
        # === Save outputs to pickle ===
        with open(f"petase_db/sequences/esm3_seq/seq_embeddings/{name}_outputs.pkl", "wb") as f:
            pickle.dump(output, f)
        with open(f"petase_db/sequences/esm3_seq/seq_embeddings/{name}_meanembeddings.pkl", "wb") as f:
            pickle.dump(all_mean_embeddings, f)
        print(f"✅ Finished {file}")

✅ Finished D1-PETase.fasta
✅ Finished TM3-PETase.fasta
✅ Finished DuraN233K.fasta
✅ Finished CaPETase_T250N.fasta
✅ Finished wtCaPETase.fasta
✅ Finished wtLCC.fasta
✅ Finished wtIsPETase_signaltrim.fasta
✅ Finished LCC_variants_tournier.fasta
✅ Finished FAST-PETase.fasta
✅ Finished IsPETase_W159H_F229Y.fasta
✅ Finished BhrPETase_variants.fasta
✅ Finished thermopetase.fasta
✅ Finished CaPETasevariants.fasta
✅ Finished HOT-PETase.fasta
✅ Finished IsPETase_S238FW159H.fasta
✅ Finished all_petase_variants.fasta
✅ Finished DuraPETase.fasta
✅ Finished TS-PETase.fasta
✅ Finished wtBhrPETase.fasta


struct embed single pdb

In [ ]:
# pip install esm huggingface_hub  (once)
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein, SamplingConfig

# 0) (one time) auth so weights can download; you can skip if already logged in
# login()  # uncomment if needed; follows the ESM README

PDB_PATH = "petase_db/structures/wtIsPETase_6ilw_Repair.pdb"
CHAIN_ID = "A"

# 1) Load structure into an ESMProtein
#    Prefer the direct helper; fallback uses ProteinChain if your wheel exposes that API.
try:
    protein = ESMProtein.from_pdb(PDB_PATH, chain_id=CHAIN_ID)
except AttributeError:
    # Fallback path used in ESM examples circulating online
    from esm.utils.structure.protein_chain import ProteinChain
    protein = ESMProtein.from_protein_chain(
        ProteinChain.from_pdb(PDB_PATH, chain_id=CHAIN_ID)
    )

# Load ESM3 open weights and encode

protein_tensor = model.encode(protein)               
# 4) Forward with embeddings requested
out = model.forward_and_sample(
    protein_tensor,
    SamplingConfig(
        return_per_residue_embeddings=True,
        return_mean_embedding=True
    )
)

import pickle
with open("esm3_output.pkl", "wb") as f:
    pickle.dump(out, f)

# 5) Grab embeddings
per_residue = out.per_residue_embedding   # shape [L, D]
mean_embed  = out.mean_embedding          # shape [D]
print(per_residue.shape, mean_embed.shape)

structure embedding (batch pdb files)

In [ ]:
#for structures
EMBEDDING_CONFIG = LogitsConfig(
    sequence=True, return_embeddings=True, return_hidden_states=True
)
for file in os.listdir("petase_db/structures/esm3_struct"):
    if file.endswith(".pdb"):
        pdb_path = os.path.join("petase_db/structures/esm3_struct", file)
        CHAIN_ID = "A"
        # 1) Load structure into an ESMProtein
        #    Prefer the direct helper; fallback uses ProteinChain if your wheel exposes that API.
        try:
            protein = ESMProtein.from_pdb(pdb_path, chain_id=CHAIN_ID)
        except AttributeError:
            # Fallback path used in ESM examples circulating online
            from esm.utils.structure.protein_chain import ProteinChain
            protein = ESMProtein.from_protein_chain(
                ProteinChain.from_pdb(pdb_path, chain_id=CHAIN_ID)
            )
        protein_tensor = model.encode(protein)
        out = model.forward_and_sample(protein_tensor,SamplingConfig(return_per_residue_embeddings=True, return_mean_embedding=True))
        per_residue = out.per_residue_embedding   # shape [L, D]
        mean_embed  = out.mean_embedding          # shape [D]

        name = os.path.splitext(file)[0]
        # === Save outputs to pickle ===
        with open(f"petase_db/structures/esm3_struct/embeddings_struct/{name}_outputs.pkl", "wb") as f:
            pickle.dump(out, f)
        with open(f"petase_db/structures/esm3_struct/embeddings_struct/{name}_meanembeddings.pkl", "wb") as f:
            pickle.dump(mean_embed, f)
        print(f"✅ Finished {file}")

Working with the fireprot.sql file

In [ ]:
import re
import pandas as pd

sql_file = "fireprotdb.sql"

# Extract amino_acids_substitution table
pattern = re.compile(
    r"INSERT INTO `amino_acids_substitution`\s*\([^)]+\)\s*VALUES\s*(.+?);",
    re.S
)
matches = pattern.findall(open(sql_file).read())

rows = []
for match in matches:
    for group in re.findall(r"\('([^']+)',\s*(\d+),\s*(\d+)\)", match):
        rows.append(group)

df = pd.DataFrame(rows, columns=["amino_acid", "from_substitutions", "to_substitutions"])
df.to_csv("amino_acids_substitution.csv", index=False)

Unifying the benchmark DBs for downstream embedding

In [ ]:
import pandas as pd
import json

# -----------------------------
# 1️⃣ FIREPROTDB
# -----------------------------
fireprot = pd.read_csv("fireprotdb_results_stability.csv")
fireprot["uniprot_id"].dropna().drop_duplicates().to_csv("uniprot_ids.txt", index=False, header=False)
fireprot["pdb_id"].dropna().drop_duplicates().to_csv("pdb_ids.txt", index=False, header=False)

# -----------------------------
# 2️⃣ MELTOME (only gene_name)
# -----------------------------
meltome = pd.read_csv("meltome_cross-species.csv")
meltome["gene_name"].dropna().drop_duplicates().to_csv("gene_names.txt", index=False, header=False)

# -----------------------------
# 3️⃣ THERMOMUTDB (JSON)
# -----------------------------
with open("thermomutdb.json", "r") as f:
    data = json.load(f)

# Prepare lists
pdb_wild = []
mutation_code = []
mutated_chain = []
uniprot = []
pdb_mutant = []

for rec in data:
    pdb_wild.append(rec.get("PDB_wild", "NA"))
    mutation_code.append(rec.get("mutation_code", "NA"))
    mutated_chain.append(rec.get("mutated_chain", "NA"))
    uniprot.append(rec.get("uniprot", "NA"))
    pdb_mutant.append(rec.get("pdb_mutant", "NA"))

# Helper: save line-by-line (preserve NA)
def save_list(values, filename):
    with open(filename, "w") as f:
        for v in values:
            f.write(str(v).strip() + "\n")

# Save each list to its own file
save_list(pdb_wild, "thermomutdb_PDB_wild.txt")
save_list(mutation_code, "thermomutdb_mutation_code.txt")
save_list(mutated_chain, "thermomutdb_mutated_chain.txt")
save_list(uniprot, "thermomutdb_uniprot.txt")
save_list(pdb_mutant, "thermomutdb_pdb_mutant.txt")

In [ ]:
#!/usr/bin/env python3
# Build a single combined list of proteins that are NOT present in crossmatch_clusters.csv.
# Also write a list of proteins that ARE present but NOT cross-matched (clusters with n_datasets == 1).
# No console prints; writes CSVs (and optional FASTA).

# ------------- CONFIG (edit) -------------
PSI_PATH      = "PSI_Biology_solubility_trainset.csv"
NESG_PATH     = "NESG_testset.csv"
PRICE_PATH    = "Price_usability_trainset.csv"
CLUSTERS_PATH = "out/crossmatch_clusters.csv"  # from crossmatch_blast_simple.py
OUTDIR        = "out"

WRITE_FASTA   = True    # also write a FASTA of missing sequences
FASTA_PATH    = "out/missing_from_crossmatch.fasta"
CSV_PATH      = "out/missing_from_crossmatch.csv"

# NEW outputs:
MISSING_SUMMARY_PATH = "out/missing_from_crossmatch_summary.txt"
NOTX_CSV_PATH        = "out/not_crossmatched.csv"              # present in clusters, but n_datasets == 1
NOTX_SUMMARY_PATH    = "out/not_crossmatched_summary.txt"
# -----------------------------------------

import os, pandas as pd

os.makedirs(OUTDIR, exist_ok=True)

def load_src(path: str, label: str) -> pd.DataFrame:
    df = pd.read_csv(path, dtype=str)
    if "sid" not in df.columns or "fasta" not in df.columns:
        raise ValueError(f"{label}: expected columns ['sid','fasta']")
    df = df[["sid","fasta"]].dropna(subset=["sid","fasta"]).copy()
    df["sid"] = df["sid"].astype(str).s tr.strip()
    df["dataset"] = label
    return df

def split_ids(s: str):
    s = (s or "").strip()
    return [x.strip() for x in s.split(";") if x.strip()] if s else []

# Load originals
psi_df   = load_src(PSI_PATH,   "PSI_Biology")
nesg_df  = load_src(NESG_PATH,  "NESG")
price_df = load_src(PRICE_PATH, "Price")

# Load clusters
cl = pd.read_csv(CLUSTERS_PATH, dtype=str).fillna("")
for col in ["psi_ids","nesg_ids","price_ids","n_datasets","cluster_id"]:
    if col not in cl.columns:
        raise ValueError(f"crossmatch_clusters.csv missing expected column: {col}")

# IDs present in clusters (per dataset)
psi_in   = set(x for ids in cl["psi_ids"]   for x in split_ids(ids))
nesg_in  = set(x for ids in cl["nesg_ids"]  for x in split_ids(ids))
price_in = set(x for ids in cl["price_ids"] for x in split_ids(ids))

# Compute missing by dataset (TRULY absent from clusters)
psi_missing   = psi_df[~psi_df["sid"].isin(psi_in)].copy()
nesg_missing  = nesg_df[~nesg_df["sid"].isin(nesg_in)].copy()
price_missing = price_df[~price_df["sid"].isin(price_in)].copy()

missing_all = pd.concat([psi_missing, nesg_missing, price_missing], ignore_index=True)
missing_all = missing_all[["dataset","sid","fasta"]].sort_values(["dataset","sid"])

# Write CSV of truly missing
missing_all.to_csv(CSV_PATH, index=False)

# Write summary of truly missing
with open(MISSING_SUMMARY_PATH, "w") as f:
    f.write("IDs present in source CSVs but NOT present in crossmatch_clusters.csv\n\n")
    f.write(f"PSI_Biology: {len(psi_missing)}\n")
    f.write(f"NESG: {len(nesg_missing)}\n")
    f.write(f"Price: {len(price_missing)}\n")
    f.write(f"\nTOTAL: {len(missing_all)}\n")

# Optional FASTA for truly missing
if WRITE_FASTA and not missing_all.empty:
    with open(FASTA_PATH, "w") as fh:
        for _, r in missing_all.iterrows():
            hdr = f"{r['dataset']}|{r['sid']}"
            seq = str(r["fasta"])
            fh.write(f">{hdr}\n")
            for i in range(0, len(seq), 80):
                fh.write(seq[i:i+80] + "\n")

# NEW PART: proteins present in clusters but NOT cross-matched (clusters with n_datasets == 1)
cl["n_datasets_int"] = pd.to_numeric(cl["n_datasets"], errors="coerce").fillna(0).astype(int)
solo = cl[cl["n_datasets_int"] == 1].copy()

rows_notx = []
for ds_col, ds_name in [("psi_ids","PSI_Biology"), ("nesg_ids","NESG"), ("price_ids","Price")]:
    for _, row in solo.iterrows():
        for sid in split_ids(row[ds_col]):
            rows_notx.append({"dataset": ds_name, "sid": sid, "cluster_id": row["cluster_id"]})

notx_df = pd.DataFrame(rows_notx).sort_values(["dataset","sid","cluster_id"])
notx_df.to_csv(NOTX_CSV_PATH, index=False)

# Summary for not-crossmatched
counts = notx_df.groupby("dataset")["sid"].nunique().reindex(["PSI_Biology","NESG","Price"]).fillna(0).astype(int)
with open(NOTX_SUMMARY_PATH, "w") as f:
    f.write("IDs present in clusters but NOT cross-matched (clusters with n_datasets == 1)\n\n")
    for ds in ["PSI_Biology","NESG","Price"]:
        f.write(f"{ds}: {int(counts.get(ds, 0))}\n")
    f.write(f"\nTOTAL: {int(counts.sum())}\n")

In [10]:
#!/usr/bin/env python3
# Adds truly missing proteins (not found in crossmatch_clusters.csv) to the output CSV and FASTA.

# ------------- CONFIG -------------
PSI_PATH      = "PSI_Biology_solubility_trainset.csv"
NESG_PATH     = "NESG_testset.csv"
PRICE_PATH    = "Price_usability_trainset.csv"
CLUSTERS_PATH = "out/crossmatch_clusters.csv"
OUTDIR        = "out"

CSV_PATH      = "out/missing_from_crossmatch.csv"
FASTA_PATH    = "out/missing_from_crossmatch.fasta"
WRITE_FASTA   = True
# ----------------------------------

import os, pandas as pd

os.makedirs(OUTDIR, exist_ok=True)

def load_src(path, label):
    df = pd.read_csv(path, dtype=str)
    if "sid" not in df.columns or "fasta" not in df.columns:
        raise ValueError(f"{label}: expected columns ['sid','fasta']")
    df = df[["sid","fasta"]].dropna().copy()
    df["sid"] = df["sid"].astype(str).str.strip()
    df["dataset"] = label
    return df

def split_ids(s):
    s = (s or "").strip()
    return [x.strip() for x in s.split(";") if x.strip()] if s else []

# Load source datasets
psi   = load_src(PSI_PATH,   "PSI_Biology")
nesg  = load_src(NESG_PATH,  "NESG")
price = load_src(PRICE_PATH, "Price")

# Load clusters (safe)
cl = pd.read_csv(CLUSTERS_PATH, dtype=str).fillna("")
for col in ["psi_ids","nesg_ids","price_ids"]:
    if col not in cl.columns:
        raise ValueError(f"{CLUSTERS_PATH} missing {col}")

# Extract IDs that appear anywhere in the cluster file
psi_in   = set(x for ids in cl["psi_ids"]   for x in split_ids(ids))
nesg_in  = set(x for ids in cl["nesg_ids"]  for x in split_ids(ids))
price_in = set(x for ids in cl["price_ids"] for x in split_ids(ids))

# Identify proteins missing from clusters
psi_missing   = psi[~psi["sid"].isin(psi_in)]
nesg_missing  = nesg[~nesg["sid"].isin(nesg_in)]
price_missing = price[~price["sid"].isin(price_in)]

missing_all = pd.concat([psi_missing, nesg_missing, price_missing], ignore_index=True)
missing_all["reason"] = "not_in_crossmatch_clusters"

# Load all proteins that *were* in clusters but not crossmatched (n_datasets==1)
if "n_datasets" in cl.columns:
    cl["n_datasets_int"] = pd.to_numeric(cl["n_datasets"], errors="coerce").fillna(0).astype(int)
    solo = cl[cl["n_datasets_int"] == 1].copy()

    def collect(col, ds_name):
        rows = []
        for _, r in solo.iterrows():
            for sid in split_ids(r[col]):
                if sid:
                    rows.append({"dataset": ds_name, "sid": sid, "fasta": None, "reason": "not_crossmatched"})
        return rows

    notx = pd.DataFrame(
        collect("psi_ids", "PSI_Biology") +
        collect("nesg_ids", "NESG") +
        collect("price_ids", "Price")
    )
else:
    notx = pd.DataFrame(columns=["dataset","sid","fasta","reason"])

# Merge: both not_crossmatched + truly missing
final_df = pd.concat([notx, missing_all], ignore_index=True)
final_df = final_df.sort_values(["dataset","sid"]).reset_index(drop=True)

# Re-attach sequences for all entries from source datasets
all_src = pd.concat([psi, nesg, price])
final_df = final_df.merge(all_src, on=["dataset","sid"], how="left", suffixes=("","_src"))
final_df["fasta"] = final_df["fasta"].fillna(final_df["fasta_src"]).dropna()

final_df[["dataset","sid","fasta","reason"]].to_csv(CSV_PATH, index=False)

# Optional FASTA output
if WRITE_FASTA:
    with open(FASTA_PATH, "w") as fh:
        for _, r in final_df.iterrows():
            seq = str(r["fasta"])
            hdr = f"{r['dataset']}|{r['sid']}|{r['reason']}"
            fh.write(f">{hdr}\n")
            for i in range(0, len(seq), 80):
                fh.write(seq[i:i+80] + "\n")

print(f"Wrote {len(final_df)} entries to {CSV_PATH}")

Wrote 11856 entries to out/missing_from_crossmatch.csv


In [ ]:
import os
#NESG

#Soluprot train

#Soluprot test 

#PRICE train

#PSI solubility

#PSI all data 5k 

#Fireprotdb.sql

#Fireprotdb results

#Thermomutdb json 

#Meltome stability

#PROTSOL huggingface



pickle loading sequence

In [ ]:
import pickle
import os
import numpy as np 
# Load the pickle file

with open("all_petase_variants_meanembeddings.pkl", "rb") as f:
    data = pickle.load(f)

# Check what it is
print(type(data))

# If it's a NumPy array
try:
    if isinstance(data, np.ndarray):
        print("Shape:", data.shape)
    else:
        print("Length:", len(data))
except Exception as e:
    print("Error checking shape:", e)

print(len(data),len(data[0]),len(data[0][0]))


pickle loading structure

In [ ]:
os.chdir("../../../structures/esm3_struct/embeddings_struct/")
with open("wtIsPETase_6ilw_Repair_25_meanembeddings.pkl", "rb") as f:
    data = pickle.load(f)

# Check what it is
print(type(data))

# If it's a NumPy array
try:
    if isinstance(data, np.ndarray):
        print("Shape:", data.shape)
    else:
        print("Length:", len(data))
except Exception as e:
    print("Error checking shape:", e)

print(len(data[0]))




fetch esm2 seq embedding

Fetch sequence embedding
This endpoint returns the ESM2 embedding vector for any protein ID that begins with MGYP. The embedding vector is obtained by averaging the final layer activations of the ESM2 transformer model over the sequence length. The model used is esm2_t36_3B_UR50D(), which underlies ESMFold v0 and v1, and produces a 2560-dimensional vector. You can pick any MGnify protein id from the MGnify protein sequence database. There are two versions of this endpoint: fetchEmbedding/ESM2/:id.json returns json format, while fetchEmbedding/ESM2/:id.bin returns a binary vector.

URL /fetchEmbedding/ESM2/:id.json

Method: GET

URL Params: id=[string]

Success Response:

Code: 200 <br />
Content: {"embedding": [...], "id": MGYP}
Sample Call:

curl "https://api.esmatlas.com/fetchEmbedding/ESM2/MGYP003323900000.json"
URL /fetchEmbedding/ESM2/:id.bin

Method: GET

URL Params: id=[string]

Success Response:

Code: 200 <br />
Content-Type: application/octet-stream
Sample Call:

curl -H "Accept: application/octet-stream" -O https://api.esmatlas.com/fetchEmbedding/ESM2/MGYP003323900000.bin
# Read the file in python, using numpy:
x = np.fromfile('MGYP003323900000.bin', dtype=np.float16)
Using python, get the embedding directly into a numpy array using the requests library:

url = "https://api.esmatlas.com/fetchEmbedding/ESM2/MGYP003323900000.bin"
header = {"Accept": "application/octet-stream"}
response = requests.get(url, headers=header)
array = np.frombuffer(response.content, dtype=np.float16)

---------------------------------------
[TO DO]
TmPredictor from Jinyuansun (novozymes comp) 
---------------------------------------
https://www.kaggle.com/code/jinyuansun/eda-and-finetune-esm

novozyme cont. https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/355209

---------------------------------------


In [ ]:
class TmPredictor(nn.Module):
    def __init__(self, esm_model):
        super().__init__()
        self.encoder = esm_model
        self.predictor = nn.Sequential(
            nn.Linear(1280,1280),
            nn.ReLU(),
            nn.Linear(1280,640),
            nn.ReLU(),
            nn.Linear(640, 1)
        )
    def forward(self, seq):
        res = self.encoder(seq, repr_layers=[33], return_contacts=False)
        rep = res['representations'][33]
        rep = rep.mean(1)
        y = self.predictor(rep)
        return y
    
net = TmPredictor(esm_model).to(device)
cal_mae = MeanAbsoluteError().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-5, weight_decay=1e-4)
train_mae = []
test_mae = []
test_rho = []
for epoch in range(4):
    for data in tqdm(train_loader):
        optimizer.zero_grad()
        seqs, pH, tm = data
        seqs = torch.tensor(seqs).long().to(device)
        tm = torch.tensor(tm.reshape(-1,1)).float().to(device)
        pred_tm = net(seqs)
        loss = torch.nn.functional.l1_loss(pred_tm, tm)
        mae = cal_mae(pred_tm, tm)
        train_mae.append(mae)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        pred_all = []
        gt_all = []
        for data in tqdm(test_loader):
            seqs, pH, tm = data
            seqs = torch.tensor(seqs).long().to(device)
            tm = torch.tensor(tm.reshape(-1,1)).float().to(device)
            pred_tm = net(seqs)
            mae = cal_mae(pred_tm, tm)
            test_mae.append(mae)
            pred_all += pred_tm.ravel().cpu().numpy().tolist()
            gt_all += tm.ravel().cpu().numpy().tolist()
        rho = stats.spearmanr(gt_all, pred_all)[0]
        test_rho.append(rho)
    print(f"Epoch: {epoch + 1}| Train MAE = {train_mae[-1]}| Test MAE = {test_mae[-1]}| Spearmanr = {rho}")



sns.lineplot(data=torch.Tensor(train_mae).numpy())
test_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/test.csv")

    pred_tm = []
for i in range(0, len(test_df), 32):
    x = test_df.protein_sequence[i: i+32].apply(convert)
    x = np.array([i for i in x])
    with torch.no_grad():
        pred = net(torch.tensor(x).to(device))
        pred_tm = pred_tm + pred.cpu().ravel().numpy().tolist()
sns.histplot(pred_tm)
test_df.insert(4, 'tm', pred_tm)
test_df.head()

---------------------------------------
[TO DO]
ESM-1v https://github.com/facebookresearch/esm/blob/main/examples/sup_variant_prediction.ipynb 
---------------------------------------

This tutorial demonstrates how to train a simple variant predictor, i.e. we predict the biological activity of mutations of a protein, using fixed embeddings from ESM. You can adopt a similar protocol to train a model for any downstream task, even with limited data.

We will use a simple classifier in sklearn (or "head" on top of the transformer features) to predict the mutation effect from precomputed ESM embeddings. The embeddings for your dataset can be dumped once using a GPU. Then, the rest of your analysis can be done on CPU.

In this particular example, we will train a model to predict the activity of ß-lactamase variants.

---------------------------------------

---------------------------------------
[TO DO] CAFA ESM2 fine-tuning (from naity) https://github.com/naity/finetune-esm
---------------------------------------

In this example, we will finetune ESM-2 for the CAFA 5 Protein Function Prediction Challenge to predict the biological function of a protein based on its primary sequence. I have already preprocessed the data and formatted the problem as a multi-class, multi-label problem. This means that for a given protein sequence, we will predict whether it is positive for each of the 100 preselected Gene Ontology (GO) terms. Thus, the target for each protein sequence is a binary vector with a length of 100.

The processed datasets can be downloaded from here. Details about the preprocessing steps can be found in the notebooks/cafa5_data_processing.ipynb notebook.

Run the following example command to finetune ESM-2 models with the processed datasets. Here, we are using the smallest model esm2_t6_8M_UR50D with 1 GPU and the LoRA approach. If you want to finetune a larger model and have multiple GPUs, please adjust num_workers and/or num-devices accordingly.

python finetune_esm/train.py \
  --experiment-name esm2_t6_8M_UR50D_lora \
  --dataset-loc data/cafa5/top100_train_split.parquet \
  --targets-loc data/cafa5/train_bp_top100_targets.npy \
  --esm-model esm2_t6_8M_UR50D \
  --num-workers 1 \
  --num-devices 1 \
  --training-mode lora \
  --learning-rate 0.0001 \
  --num-epochs 5

mlflow server --host 127.0.0.1 --port 8080 --backend-store-uri ./finetune_results/mlflow

*note: model = FinetuneESM(esm_model=esm_model, dropout_p=dropout_p, num_classes=num_classes)

	•	FinetuneESM wraps a pretrained ESM-2 transformer from Hugging Face.
	•	It replaces the final classification head with a new one sized to your task (num_classes=100 for CAFA).
	•	The pretrained weights of ESM are loaded as the starting point.

If you want distributed scaling or MLflow logging:

	•	Replace FinetuneESM with RegressionHead
	•	Remove tokenizer, collate_fn, and ESM loading
	•	Pass your pre-embedded arrays instead of sequences
	•	Switch the loss to MSELoss (for regression) or BCEWithLogitsLoss (if binary 0/1 classification)

Example of this applied to our 150 PETase dataset:

---------------------------------------


In [ ]:
import numpy as np

X_seq = np.load("esm3_seq_embeddings.npy")
X_struct = np.load("esm3_struct_embeddings.npy")
X = np.concatenate([X_seq, X_struct], axis=1)  # shape (150, 2560)
y = np.load("labels_Tm.npy")  # or solubility

import torch
import torch.nn as nn
import lightning as L

class RegressionHead(L.LightningModule):
    def __init__(self, input_dim=2560, hidden_dim=512, lr=1e-4):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # output between 0–1
        )
        self.lr = lr
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, _):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, _):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                         torch.tensor(y_train, dtype=torch.float32))
val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                       torch.tensor(y_val, dtype=torch.float32))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
model = RegressionHead(input_dim=X.shape[1], lr=1e-4)
trainer = L.Trainer(max_epochs=100, accelerator="gpu", devices=1)
trainer.fit(model, train_loader, val_loader)  

SyntaxError: invalid syntax (3373901559.py, line 1)

-----

[TO DO] OBTAIN FUNCTIONAL ANNOTAITON OF PETASE DATASET

[TO DO] MLP HEAD ON PETASE DATASET